In [1]:
sc

org.apache.spark.SparkContext@a02bd315

In [2]:
val sqlContext = new org.apache.spark.sql.SQLContext(sc)

In [7]:
val df=sqlContext.read.parquet("MGH-100P-CENTERS/KMeansModel_13000K/data")

In [8]:
df.printSchema()

root
 |-- clusterIdx: integer (nullable = false)
 |-- clusterCenter: struct (nullable = true)
 |    |-- type: byte (nullable = false)
 |    |-- size: integer (nullable = true)
 |    |-- indices: array (nullable = true)
 |    |    |-- element: integer (containsNull = false)
 |    |-- values: array (nullable = true)
 |    |    |-- element: double (containsNull = false)



In [9]:
df.head()

[8013,[1,null,null,WrappedArray(-1.8897856191312986E12, -3.2904972946708705E7, 1214696.0289277907, 8309715.48823891, 1.2740925491225444E7, -1.848451839562781E7, -1900500.2205226824, 5278087.319497016, -3977205.2867267863, -2810883.575134812, -418938.95277429966, -3050707.0862161266, 1127623.802829448, 2970740.4906033985, 2328647.753912008, 553471.913679801, 3190389.0271659964, 1739591.649945637, 777869.7661639026, -649409.5838782997, 213971.54394377238, 200974.405278543, -306920.93379156047, 561157.5727150417, -994143.2007966664, 114065.3915987016, 299883.2990017004, -124613.68212105532, -312473.5806767948, 70770.57815804916, -163170.30650063095, 157018.3729711284, 52221.46030551026, 235839.92983651883, -12597.125693008204, -57101.07586979955, 10409.1442...

In [8]:
df.count() //old value

24762

In [10]:
df.count() //100P

13000

In [11]:
val data1 = df.select("clusterCenter.values")
data1.getClass().getName()

org.apache.spark.sql.DataFrame

In [12]:
data1.printSchema()

root
 |-- values: array (nullable = true)
 |    |-- element: double (containsNull = false)



In [12]:
val data1RDD=data1.rdd
data1RDD.take(2)

Array([WrappedArray(-1.8897856191312986E12, -3.2904972946708705E7, 1214696.0289277907, 8309715.48823891, 1.2740925491225444E7, -1.848451839562781E7, -1900500.2205226824, 5278087.319497016, -3977205.2867267863, -2810883.575134812, -418938.95277429966, -3050707.0862161266, 1127623.802829448, 2970740.4906033985, 2328647.753912008, 553471.913679801, 3190389.0271659964, 1739591.649945637, 777869.7661639026, -649409.5838782997, 213971.54394377238, 200974.405278543, -306920.93379156047, 561157.5727150417, -994143.2007966664, 114065.3915987016, 299883.2990017004, -124613.68212105532, -312473.5806767948, 70770.57815804916, -163170.30650063095, 157018.3729711284, 52221.46030551026, 235839.92983651883, -12597.125693008204, -57101.07586979955, 10409.144252873...

In [13]:
val data1Double = data1.map(r=>r.getList(0))
data1Double.take(2)

Array([-1.8897856191312986E12, -3.2904972946708705E7, 1214696.0289277907, 8309715.48823891, 1.2740925491225444E7, -1.848451839562781E7, -1900500.2205226824, 5278087.319497016, -3977205.2867267863, -2810883.575134812, -418938.95277429966, -3050707.0862161266, 1127623.802829448, 2970740.4906033985, 2328647.753912008, 553471.913679801, 3190389.0271659964, 1739591.649945637, 777869.7661639026, -649409.5838782997, 213971.54394377238, 200974.405278543, -306920.93379156047, 561157.5727150417, -994143.2007966664, 114065.3915987016, 299883.2990017004, -124613.68212105532, -312473.5806767948, 70770.57815804916, -163170.30650063095, 157018.3729711284, 52221.46030551026, 235839.92983651883, -12597.125693008204, -57101.07586979955, 10409.144252873892, -1109.140...

In [14]:
import org.apache.spark.sql.functions.explode
val data2 = data1.select(explode(data1("values")).alias("b_columns"))

In [15]:
data2.show()

+--------------------+
|           b_columns|
+--------------------+
|-1.88978561913129...|
|-3.29049729467087...|
|  1214696.0289277907|
|    8309715.48823891|
|1.2740925491225444E7|
|-1.848451839562781E7|
| -1900500.2205226824|
|   5278087.319497016|
| -3977205.2867267863|
|  -2810883.575134812|
| -418938.95277429966|
| -3050707.0862161266|
|   1127623.802829448|
|  2970740.4906033985|
|   2328647.753912008|
|    553471.913679801|
|  3190389.0271659964|
|   1739591.649945637|
|   777869.7661639026|
|  -649409.5838782997|
+--------------------+
only showing top 20 rows



In [16]:
val data3=data2.rdd
data3.take(5)

Array([-1.8897856191312986E12], [-3.2904972946708705E7], [1214696.0289277907], [8309715.48823891], [1.2740925491225444E7])

In [17]:
data3.count()

2476200

In [18]:
%AddJar  https://github.com/kartha01/spark-tsne/raw/master/bin/spark-tsne-core_2.10-0.1-SNAPSHOT.jar

Using cached version of spark-tsne-core_2.10-0.1-SNAPSHOT.jar


In [19]:
import com.github.saurfang.spark.tsne.impl._
import com.github.saurfang.spark.tsne.tree.SPTree
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.mllib.linalg.distributed.RowMatrix
import org.apache.spark.mllib.stat.{MultivariateStatisticalSummary, Statistics}

In [20]:
val data3doubVals = data3.map{ row =>   row.getDouble(0) }
data3doubVals.getClass().getName()
data3doubVals.take(2)

Array(-1.8897856191312986E12, -3.2904972946708705E7)

In [21]:
val mean = data3doubVals.mean()
val std = data3doubVals.stdev()

In [22]:
mean

-3.0007358447105064E10

In [23]:
std

3.5859201782176404E11

In [24]:
data1Double.take(2).foreach(println)

[-1.8897856191312986E12, -3.2904972946708705E7, 1214696.0289277907, 8309715.48823891, 1.2740925491225444E7, -1.848451839562781E7, -1900500.2205226824, 5278087.319497016, -3977205.2867267863, -2810883.575134812, -418938.95277429966, -3050707.0862161266, 1127623.802829448, 2970740.4906033985, 2328647.753912008, 553471.913679801, 3190389.0271659964, 1739591.649945637, 777869.7661639026, -649409.5838782997, 213971.54394377238, 200974.405278543, -306920.93379156047, 561157.5727150417, -994143.2007966664, 114065.3915987016, 299883.2990017004, -124613.68212105532, -312473.5806767948, 70770.57815804916, -163170.30650063095, 157018.3729711284, 52221.46030551026, 235839.92983651883, -12597.125693008204, -57101.07586979955, 10409.144252873892, -1109.140789024793, -32087.213374926232, 35326.280811316596, -20675.368126883925, 45786.831570610986, -20303.89514609269, 1163.309258838132, -10309.582833075998, 3051.5902487616995, -5365.314856526558, 6430.9858118775, 2804.8685882658015, 67.0218221012101, 

In [25]:
val data1DoubleWithIdx=data1Double.zipWithIndex()
data1DoubleWithIdx.take(2).foreach(println)

([-1.8897856191312986E12, -3.2904972946708705E7, 1214696.0289277907, 8309715.48823891, 1.2740925491225444E7, -1.848451839562781E7, -1900500.2205226824, 5278087.319497016, -3977205.2867267863, -2810883.575134812, -418938.95277429966, -3050707.0862161266, 1127623.802829448, 2970740.4906033985, 2328647.753912008, 553471.913679801, 3190389.0271659964, 1739591.649945637, 777869.7661639026, -649409.5838782997, 213971.54394377238, 200974.405278543, -306920.93379156047, 561157.5727150417, -994143.2007966664, 114065.3915987016, 299883.2990017004, -124613.68212105532, -312473.5806767948, 70770.57815804916, -163170.30650063095, 157018.3729711284, 52221.46030551026, 235839.92983651883, -12597.125693008204, -57101.07586979955, 10409.144252873892, -1109.140789024793, -32087.213374926232, 35326.280811316596, -20675.368126883925, 45786.831570610986, -20303.89514609269, 1163.309258838132, -10309.582833075998, 3051.5902487616995, -5365.314856526558, 6430.9858118775, 2804.8685882658015, 67.0218221012101,

In [26]:
val scaledData = data1DoubleWithIdx.map(x => Vectors.dense(x._1.toArray().map(v => (v.asInstanceOf[Double] - mean) / std))).cache()
scaledData.take(2).foreach(println)

[-5.186334799032211,0.08358929363859118,0.08368444263042567,0.083704228401181,0.0837165856478088,0.0836295077366034,0.08367575533094708,0.08369577414671332,0.08366996405572902,0.0836732165590019,0.08367988693788231,0.08367254776689509,0.08368419981345884,0.08368933968438784,0.08368754909032891,0.08368259868498801,0.08368995221485602,0.0836859063987053,0.08368322445979985,0.08367924422800688,0.08368165192557099,0.08368161568065192,0.08368019932079496,0.08368262011786619,0.08367828287471459,0.08368137331883303,0.08368189150635023,0.08368070771820095,0.08368018383621469,0.08368125258325809,0.08368060019594038,0.08368149310114618,0.08368120085562074,0.08368171290959965,0.083681020097035,0.08368089598955919,0.08368108425426328,0.08368105213340038,0.08368096574532968,0.08368115374029565,0.08368099756936559,0.08368118291147136,0.08368099860528654,0.08368105847054659,0.0836810264762703,0.08368106373636651,0.08368104026427375,0.08368107316043452,0.08368106304833764,0.08368105541334682,0.0836810

In [27]:
scaledData.count()

13000

In [ ]:
// non scaling
val scaledData = data1DoubleWithIdx.map(x => Vectors.dense(x._1.toArray().map(v => v.asInstanceOf[Double]))).cache()
scaledData.take(2).foreach(println)


In [28]:

val labels = data1DoubleWithIdx.map(_._2).collect()
val matrix = new RowMatrix(scaledData)

// PCA again?
//val pcaMatrix = matrix.multiply(matrix.computePrincipalComponents(50))
val pcaMatrix=matrix
pcaMatrix.rows.cache()

MapPartitionsRDD[36] at mapPartitions at RowMatrix.scala:429

In [31]:
labels

Array(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, ...

In [41]:
import java.io.{BufferedWriter, OutputStreamWriter,FileWriter}

import org.apache.hadoop.fs.{FileSystem, Path}

import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.SparkContext._
import org.slf4j.LoggerFactory

// FileWriter
//val file = new File("MGH-TEST-100/100PtSNE_res.csv")
val writer = new BufferedWriter(new FileWriter("MGH-TEST-100/100PtSNE_res.csv"))
writer.write("tsne outputs")
writer.close()

// FileWriter
//val file2 = new File("MGH-TEST-100/100PtSNE_cost.csv")
val costWriter = new BufferedWriter(new FileWriter("MGH-TEST-100/100PtSNE_cost.csv"))
costWriter.write("tsne outputs")
costWriter.close()

In [43]:
val hadoopConf = sc.hadoopConfiguration
val fs = FileSystem.get(hadoopConf)
   //val costWriter = new BufferedWriter(new OutputStreamWriter(fs.create(new Path(s"MGH-100P-CENTERS/7Pt-SNE/cost.txt"), true)))
    val costWriter = new BufferedWriter(new FileWriter("MGH-TEST-100/100PtSNE_cost.csv"))


    //SimpleTSNE.tsne(pcaMatrix, perplexity = 20, maxIterations = 200)
    BHTSNE.tsne(pcaMatrix, maxIterations = 500, callback = {
    //LBFGSTSNE.tsne(pcaMatrix, perplexity = 10, maxNumIterations = 500, numCorrections = 10, convergenceTol = 1e-8)
      case (i, y, loss) =>
        if(loss.isDefined) println(s"$i iteration finished with loss $loss")

        //val os = fs.create(new Path(s"MGH-100P-CENTERS/7Pt-SNE/result${"%05d".format(i)}.csv"), true)
        //val writer = new BufferedWriter(new OutputStreamWriter(os))
        val writer = new BufferedWriter(new FileWriter("MGH-TEST-100/100PtSNE_res.csv"))
        try {
          (0 until y.rows).foreach {
            row =>
              writer.write(labels(row).toString)
              writer.write(y(row, ::).inner.toArray.mkString(",", ",", "\n"))
              //writer.write(y(row, ::).inner.toArray.mkString(, ",", "\n"))
          }
          //if(loss.isDefined) costWriter.write(loss.get + "\n")
          try {
                costWriter.write(loss.get + "\n")
          } catch {
                case e: Exception => costWriter.write("No losses ("+None+") \n")
          }
          //costWriter.write(loss.get + "\n")  
        } finally {
          writer.close()
        }
    })
    costWriter.close()

10 iteration finished with loss Some(0.0025321755621685042)
20 iteration finished with loss Some(0.0025321755307195536)
30 iteration finished with loss Some(0.002532165719908042)
40 iteration finished with loss Some(0.002496761107968782)
50 iteration finished with loss Some(0.0019184507900249232)
60 iteration finished with loss Some(0.001638838727397123)
70 iteration finished with loss Some(0.0014935913471999407)
80 iteration finished with loss Some(0.0013920183682254986)
90 iteration finished with loss Some(0.0013137970259043303)
100 iteration finished with loss Some(0.0012505115523237383)
110 iteration finished with loss Some(1.768690430336841E-4)
120 iteration finished with loss Some(1.6271828897015836E-4)
130 iteration finished with loss Some(1.5126205731273956E-4)
140 iteration finished with loss Some(1.410793834051636E-4)
150 iteration finished with loss Some(1.3182834254434628E-4)
160 iteration finished with loss Some(1.2343228116694355E-4)
170 iteration finished with loss Some(